# Convert FIRE data

This script will convert the FIRE .hdf5 files into a .json data files that can be read into Firefly. 

FIREreader is the class that will allow you to read in files within a directory, create the dictionary, and write out the json files


In [46]:
%load_ext autoreload
%autoreload 2

from firefly_api.FIREreader import FIREreader
import numpy as np
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Set the defaults and create the .json files

In [137]:
reader = FIREreader(
    #snapdir = "/Users/agurvich/research/snaps/Control_G4_20/snapdir_050/",
    snapdir = "/Users/ageller/Visualizations/FIREdata/snapdir_050/",
    snapnum = 50,
    ptypes=['PartType0','PartType4'],
    UInames=['Gas','Stars'],
    dec_factors=[10,100],
    returnKeys=['Density','Velocities','Temperature'],
    doMags=[0,0,0],
    doLogs=[1,0,1],
    filterFlags=[1,1,1], ## Note: previously Velocities were automatically included, now you must mark 1 to filter 
    colormapFlags=[1,1,1], 
    JSONdir='/Users/ageller/Visualizations/Firefly/static/data/',
    doValidate=False #Added this in to FIREreader.py and reader.py
)
reader.loadData()

reader.options['color']['Gas']=[1,0,0,1]
reader.options['color']['Stars']=[0,0,1,1]
reader.options['sizeMult']['Gas']=0.3
reader.options['sizeMult']['Stars']=0.3

#reader.dumpToJSON()

outDict = {"parts":{}, "options":None}
for particleGroup in reader.particleGroups:
    if (particleGroup.UIname == "Gas"):
        outDict['parts'][particleGroup.UIname] = particleGroup.outputToDict(np.arange(1000))
outDict['options'] = reader.options.outputToDict()


Loading ptype PartType0
/Users/ageller/Visualizations/FIREdata/snapdir_050/snapshot_050.0.hdf5
Loading ptype PartType4
/Users/ageller/Visualizations/FIREdata/snapdir_050/snapshot_050.0.hdf5


In [126]:
#print(outDict)

## Send this data to the Flask app

In [102]:
import sys
import json
import requests
from IPython.display import IFrame

*This sends the data.*

In [139]:
s = pd.Series(outDict).to_json()
requests.post("http://localhost:5000/data_input", json=s)

<Response [200]>

*This shows the viewer.*

In [104]:
url = "http://localhost:5000/combined"
IFrame(url, width=700, height=300)